# Load packages

In [1]:
!pip uninstall -y kfp
!pip install kfp

Found existing installation: kfp 1.4.0
Uninstalling kfp-1.4.0:
  Successfully uninstalled kfp-1.4.0
  Using cached kfp-1.4.0-py3-none-any.whl


In [2]:
import kfp
import json
import os
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler
from pathlib import Path

kfp.__version__

'1.4.0'

# Enter your gateway loadbalancer and the token from the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

In [3]:
#Update values for the load balancer and auth session
HOST='istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh"
namespace="kubeflow-user-example-com"

In [4]:
client = kfp.Client(host=HOST+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

### Using dsl for model archiver and torchserve integration
#### Init container is used for mar file generation

In [5]:
DEPLOY="bertserve"
MODEL="bert"

In [6]:
prepare_tensorboard_op = load_component_from_file("./examples/bert/yaml/tensorboard/component.yaml")
prep_op = components.load_component_from_file(
    "./examples/bert/yaml/pre_process/component.yaml"
)
train_op = components.load_component_from_file(
    "./examples/bert/yaml/train/component.yaml"
)
deploy_op = load_component_from_file("./examples/bert/yaml/deploy/component.yaml")

In [7]:
minio_op = components.load_component_from_file(
    "./examples/bert/yaml/minio/component.yaml"
)

In [8]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert(minio_endpoint='minio-service.kubeflow:9000',
    log_bucket='mlpipeline',
    log_dir=f'tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}',
    mar_path=f'mar/{dsl.RUN_ID_PLACEHOLDER}/model-store',
    config_prop_path=f'mar/{dsl.RUN_ID_PLACEHOLDER}/config',
    model_uri = f's3://mlpipeline/mar/{dsl.RUN_ID_PLACEHOLDER}',
    tf_image='jagadeeshj/tb_pluign:v1.8'):

    @dsl.component
    def ls(input_dir: str):
        return dsl.ContainerOp(
            name='list',
            image='busybox:latest',
            command=["ls", "-R", "%s" % input_dir]
        )
    
    prepare_tb_task = prepare_tensorboard_op(
        log_dir_uri=f's3://{log_bucket}/{log_dir}',
        image=tf_image,
        pod_template_spec=json.dumps({
            'spec': {
                'containers': [{
                    'env': [{
                        'name': 'AWS_ACCESS_KEY_ID',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'accesskey'
                            }
                        }
                    }, {
                        'name': 'AWS_SECRET_ACCESS_KEY',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'secretkey'
                            }
                        }
                    }, {
                        'name': 'AWS_REGION',
                        'value': 'minio'
                    }, {
                        'name': 'S3_ENDPOINT',
                        'value': f'{minio_endpoint}',
                    }, {
                        'name': 'S3_USE_HTTPS',
                        'value': '0',
                    }, {
                        'name': 'S3_VERIFY_SSL',
                        'value': '0',
                    }]
                }],
            },
        })
    ).set_display_name("Visualization")
    
    prep_task=prep_op().after(prepare_tb_task).set_display_name("Preprocess & Transform")
    train_task=train_op(input_data=prep_task.outputs['output_data'], profiler="pytorch").after(prep_task).set_display_name("Training")
    
    minio_tb_upload = minio_op(bucket_name="mlpipeline", folder_name=log_dir, input_path=train_task.outputs['tensorboard_root'], filename="").after(train_task).set_display_name("Tensorboard Events Pusher")
    minio_mar_upload = minio_op(bucket_name="mlpipeline", folder_name=mar_path, input_path=train_task.outputs['checkpoint_dir'], filename="bert_test.mar").after(train_task).set_display_name("Mar Pusher")
    minio_config_upload = minio_op(bucket_name="mlpipeline", folder_name=config_prop_path, input_path=train_task.outputs['checkpoint_dir'], filename="config.properties").after(train_task).set_display_name("Conifg Pusher")
    
    
    model_uri= str(model_uri)
    isvc_yaml = '''
    apiVersion: "serving.kubeflow.org/v1beta1"
    kind: "InferenceService"
    metadata:
      name: {}
      namespace: {}
    spec:
      predictor:
        serviceAccountName: sa
        pytorch:
          storageUri: {}
          resources:
            limits:
              memory: 4Gi   
    '''.format(DEPLOY, namespace, model_uri)
    deploy_task = deploy_op(
        action='apply',
        inferenceservice_yaml=isvc_yaml
    ).after(minio_mar_upload).set_display_name("Deployer")
    

In [9]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz', type_check=True)

In [10]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [94]:
!kubectl get isvc $DEPLOY

NAME        URL                                                      READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                 AGE
bertserve   http://bertserve.kubeflow-user-example-com.example.com   True           100                              bertserve-predictor-default-b82pm   22h


# Inference

In [80]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

bertserve.kubeflow-user-example-com.example.com


In [81]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

'bertserve.kubeflow-user-example-com.example.com'

# Prediction

In [11]:
!curl -v -H "Host: bertserve.kubeflow-user-example-com.example.com" -H "Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh" "http://istio-ingressgateway.istio-system.svc.cluster.local/v1/models/bert:predict" -d @./examples/bert/sample.txt

*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 84 out of 84 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 33
< content-type: application/json; charset=UTF-8
< date: Thu, 06 May 2021 17:59:42 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 291
< 
* Connection #0 to host istio-ingressgateway.istio-system.svc.cluster.local left intact
{"predictions": ["\"Business\""]}

In [12]:
!curl -v -H "Host: bertserve.kubeflow-user-example-com.example.com" -H "Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh" "http://istio-ingressgateway.istio-system.svc.cluster.local/v1/models/bert:explain" -d @./examples/bert/sample.txt

*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:explain HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 84 out of 84 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 322
< content-type: application/json; charset=UTF-8
< date: Thu, 06 May 2021 17:59:57 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 16371
< 
* Connection #0 to host istio-ingressgateway.istio-system.svc.cluster.local left intact
{"explanations": [{"words": ["[CLS]", "bloomberg", "has", "reported", "on", "the", "ec